# Etapa 4: Adicionar um pipeline de CI/CD de implantação de modelos


Nesta etapa, você cria um pipeline de CI/CD automatizado para construção de modelos usando [Amazon SageMaker Projects](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects.html).

![](img/seis-etapas-4.png)

Você vai usar um [template de projeto MLOps fornecido pelo SageMaker para construção e treinamento de modelos](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-templates-sm.html#sagemaker-projects-templates-code-commit) para provisionar uma automação de fluxo de trabalho de CI/CD com [AWS CodePipeline](https://aws.amazon.com/codepipeline/) e um repositório de código [AWS CodeCommit](https://aws.amazon.com/codecommit/).

Os templates de projeto do SageMaker oferecem as seguintes opções de repositórios de código, ferramentas de automação de fluxo de trabalho e estágios de pipeline:
- **Repositório de código**: AWS CodeCommit ou repositórios Git de terceiros como GitHub e Bitbucket
- **Automação de fluxo de trabalho de CI/CD**: AWS CodePipeline ou Jenkins
- **Estágios do pipeline**: Construção e treinamento de modelos, implantação de modelos ou ambos

<div class="alert alert-info"> Certifique-se de estar usando o kernel <code>Python 3</code> no JupyterLab para este notebook.</div>

In [1]:
import boto3
import sagemaker 
from time import gmtime, strftime, sleep

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Fetched defaults config from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
%store -r 

%store

try:
    initialized
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERRO] VOCÊ PRECISA EXECUTAR o notebook 00-inicie-aqui")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")


Stored variables and their in-db values:
baseline_s3_url                         -> 's3://sagemaker-us-east-1-533267005474/from-idea-t
bucket_name                             -> 'sagemaker-us-east-1-533267005474'
bucket_prefix                           -> 'from-idea-to-prod/xgboost'
dataset_feature_group_name              -> 'from-idea-to-prod-15-02-45-15'
dataset_file_local_path                 -> 'data/bank-additional/bank-additional-full.csv'
domain_id                               -> 'd-j2g52sky4w9s'
evaluation_s3_url                       -> 's3://sagemaker-us-east-1-533267005474/from-idea-t
experiment_name                         -> 'from-idea-to-prod-experiment-15-03-26-12'
feature_store_bucket_prefix             -> 'from-idea-to-prod/feature-store'
initialized                             -> True
input_s3_url                            -> 's3://sagemaker-us-east-1-533267005474/from-idea-t
mlflow_arn                              -> 'arn:aws:sagemaker:us-east-1:533267005474:mlflow

In [3]:
%load_ext autoreload
%autoreload 2

## Criar um projeto MLOps
⭐ Você pode criar um projeto programaticamente neste notebook - **Opção 1** ou na interface do Studio - **Opção 2**.

A Opção 1 é recomendada, pois não requer entrada manual e não tem dependência da interface do usuário.</br>
A Opção 2 é fornecida para demonstrar o [fluxo da interface do usuário **Criar Projeto**](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-create.html).

### Opção 1: Criar projeto programaticamente
Nesta seção, você usa o `boto3` para criar um projeto MLOps através da API do SageMaker.

In [4]:
sm = boto3.client("sagemaker")
sc = boto3.client("servicecatalog")

sc_provider_name = "Amazon SageMaker"
sc_product_name = "MLOps template for model building and training"

In [5]:
p_ids = [p['ProductId'] for p in sc.search_products(
    Filters={
        'FullTextSearch': [sc_product_name]
    },
)['ProductViewSummaries'] if p["Name"]==sc_product_name]

In [6]:
p_ids

['prod-53ibyqbj2cgmo']

In [7]:
# Se você receber qualquer exceção deste código, vá para a Opção 2 e crie um projeto na interface do Studio
if not len(p_ids):
    raise Exception("Nenhum produto Amazon SageMaker MLOps encontrado!")
elif len(p_ids) > 1:
    raise Exception("Muitos produtos Amazon SageMaker MLOps correspondentes encontrados!")
else:
    product_id = p_ids[0]
    print(f"ID do produto ML Ops: {product_id}")

ID do produto ML Ops: prod-53ibyqbj2cgmo


In [8]:
provisioning_artifact_id = sorted(
    [i for i in sc.list_provisioning_artifacts(
        ProductId=product_id
    )['ProvisioningArtifactDetails'] if i['Guidance']=='DEFAULT'],
    key=lambda d: d['Name'], reverse=True)[0]['Id']

In [9]:
provisioning_artifact_id

'pa-dxonbcmsdfyq6'

In [10]:
project_name = f"model-build-{strftime('%m-%d-%H-%M-%S', gmtime())}"
project_parameters = [] # Este modelo de projeto integrado do SageMaker não tem parâmetros

Por fim, crie um projeto SageMaker a partir do modelo de produto do catálogo de serviços:

In [11]:
# criar projeto SageMaker
r = sm.create_project(
    ProjectName=project_name,
    ProjectDescription="Model build project",
    ServiceCatalogProvisioningDetails={
        'ProductId': product_id,
        'ProvisioningArtifactId': provisioning_artifact_id,
    },
)

print(r)
project_id = r["ProjectId"]

{'ProjectArn': 'arn:aws:sagemaker:us-east-1:533267005474:project/model-build-10-15-04-05-48', 'ProjectId': 'p-lcexwhfupbld', 'ResponseMetadata': {'RequestId': '2f19a1ab-ed45-4b85-8470-cd2e43679741', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '2f19a1ab-ed45-4b85-8470-cd2e43679741', 'content-type': 'application/x-amz-json-1.1', 'content-length': '121', 'date': 'Tue, 15 Oct 2024 04:05:48 GMT'}, 'RetryAttempts': 0}}


<div class="alert alert-info"> 💡 <strong> Aguarde até que a criação do projeto seja concluída executando a próxima célula</strong>
</div>



<svg width="800" height="125" viewBox="0 0 800 125" xmlns="http://www.w3.org/2000/svg">
    <defs>
        <linearGradient id="fadeGradient" x1="0" x2="1">
            <stop offset="0%" stop-color="#F0F0F0"/>
            <stop offset="100%" stop-color="#F0F0F0" stop-opacity="0"/>
        </linearGradient>
        <mask id="fadeMask">
            <rect x="0" y="0" width="750" height="125" fill="white"/>
            <rect x="750" y="0" width="50" height="125" fill="url(#fadeGradient)"/>
        </mask>
    </defs>
    <path d="M3,50 A50,50 0 0 1 53,3 L797,3 L797,97 L97,97 L50,115 L3,97 Z" fill="#F0F0F0" stroke="#E0E0E0" stroke-width="1" mask="url(#fadeMask)"/>
    <circle cx="50" cy="50" r="30" fill="#57c4f8" stroke="#57c4f8" stroke-width="1"/>
    <circle cx="50" cy="50" r="25" fill="#F0F0F0"/>
    <line x1="50" y1="50" x2="50" y2="30" stroke="#57c4f8" stroke-width="3" stroke-linecap="round"/>
    <line x1="50" y1="50" x2="65" y2="50" stroke="#57c4f8" stroke-width="3" stroke-linecap="round"/>
    <text x="100" y="34" font-family="Arial, sans-serif" font-size="14" fill="#333333">A próxima célula pode levar alguns minutos para executar.</text>
</svg>

In [12]:
# A criação do projeto leva de 3 a 5 minutos
while sm.describe_project(ProjectName=project_name)['ProjectStatus'] != 'CreateCompleted':
    print("Aguardando a conclusão da criação do projeto")
    sleep(10)
    
print(f"Criação do projecto MLOps {project_name} completado")

Aguardando a conclusão da criação do projeto
Aguardando a conclusão da criação do projeto
Aguardando a conclusão da criação do projeto
Aguardando a conclusão da criação do projeto
Aguardando a conclusão da criação do projeto
Aguardando a conclusão da criação do projeto
Aguardando a conclusão da criação do projeto
Aguardando a conclusão da criação do projeto
Aguardando a conclusão da criação do projeto
Aguardando a conclusão da criação do projeto
Aguardando a conclusão da criação do projeto
Criação do projecto MLOps model-build-10-15-04-05-48 completado


### Fim da Opção 1: Criar projeto programaticamente
Agora você provisionou um modelo de projeto em seu ambiente SageMaker. Navegue até a seção **Configurar o projeto MLOps**.

Opção 2: Criar um projeto na interface do Studio
<div class="alert alert-info"> 💡 <strong> Pule esta seção se você criou um projeto programaticamente </strong>

Siga as instruções no Guia do Desenvolvedor – [Criar um Projeto MLOps usando Amazon SageMaker Studio ou Studio Classic](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-create.html). Escolha a opção **Studio**.

Para o modelo, escolha **Construção e treinamento de modelo**.
Nos **Detalhes do projeto**, você precisa fornecer um nome e uma descrição opcional do projeto. Este modelo não tem nenhum parâmetro.

Escolha **Criar** e aguarde até que o projeto apareça na lista de Projetos.

### Resolver problemas com a criação do projeto

#### Processo de criação do projeto preso em pendente
Se após 5 minutos o banner de criação do projeto ainda estiver ativo, feche a janela do navegador do Studio e faça login no Studio novamente.

![](img/project-creation-pending.png)

#### Mensagens de erro
❗ Se você ver uma mensagem de erro semelhante a:
```
Seu projeto não pôde ser criado
O Studio encontrou um erro ao criar seu projeto. Tente criar o projeto novamente.

O CodeBuild não está autorizado a realizar: sts:AssumeRole em arn:aws:iam::XXXX:role/service-role/AmazonSageMakerServiceCatalogProductsCodeBuildRole (Serviço: AWSCodeBuild; Código de Status: 400; Código de Erro: InvalidInputException; ID da Solicitação: 4cf59a54-0c59-476a-a970-0ac656db4402; Proxy: null)
```

veja os passos 5-6 de [Permissões do SageMaker Studio Necessárias para Usar Projetos](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-studio-updates.html). Certifique-se de que você tem todas as funções de projeto necessárias listadas no cartão **Apps** sob **Projects**.

Alternativamente, você pode criar as funções necessárias usando o modelo CloudFormation fornecido [`cfn-templates/sagemaker-project-templates-roles.yaml`](cfn-templates/sagemaker-project-templates-roles.yaml).
Execute no diretório clone do repositório a partir do terminal de linha de comando onde você tem as permissões correspondentes:

```sh
aws cloudformation deploy \
    --template-file cfn-templates/sagemaker-project-templates-roles.yaml \
    --stack-name sagemaker-project-template-roles \
    --capabilities CAPABILITY_IAM CAPABILITY_NAMED_IAM \
    --parameter-overrides \
    CreateCloudFormationRole=YES \
    CreateCodeBuildRole=YES \
    CreateCodePipelineRole=YES \
    CreateEventsRole=YES \
    CreateProductsExecutionRole=YES 

### Fim da Opção 2: Criar um projeto na interface do Studio
Agora que você criou o projeto, passe para a seção **Configurar o projeto MLOps**.

---


## Configurar o projeto MLOps
O projeto executa automaticamente um pipeline de construção de modelo padrão fornecido assim que é criado. Este pipeline é um exemplo de espaço reservado no projeto para seu próprio pipeline personalizado. Ignore o pipeline padrão por enquanto.
Os modelos de projeto implantam a seguinte arquitetura em sua conta AWS:

![](img/mlops-model-build-train.png)


Os principais componentes são:
1. O modelo de projeto é disponibilizado através dos Projetos SageMaker e do portfólio do AWS Service Catalog
2. Um pipeline do CodePipeline com dois estágios - `Source` para baixar o código-fonte de um repositório CodeCommit e `Build` para criar e executar um pipeline SageMaker
3. Um pipeline SageMaker padrão com workflow (fluxo de trabalho) de construção, treinamento e registro de modelo (model build, train, and register workflow)
4. Um repositório de código inicial no CodeCommit com uma versão padrão fornecida de um código de espaço reservado

Este projeto contém todo o código necessário e a infraestrutura para implementar um pipeline de CI/CD automatizado a partir de um modelo predefinido.
Para começar a usar o projeto com seu pipeline, você precisa completar os seguintes passos:
1. Clonar o repositório CodeCommit do projeto para o volume EBS do seu notebook
2. Substituir o código de amostra do modelo de pipeline ML pelo seu código real de construção do pipeline, conforme implementado no notebook do passo 3
3. Modificar o arquivo `codebuild-buildspec.yml` para referenciar o nome correto do módulo Python e definir os parâmetros do projeto

As próximas seções guiarão você através destes passos. Para instruções detalhadas e um exemplo prático, consulte o guia de desenvolvimento [Passo a Passo do Projeto MLOps do SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-walkthrough.html).

Se você usou a opção 1 `boto3` para criar um projeto MLOps, o `project_name` e o `project_id` são definidos automaticamente. Você pode executar a seguinte célula de código para imprimir os valores. Se você seguiu as instruções da interface do usuário para criar um projeto, você deve definir o `project_name` manualmente.

In [13]:
try:
    print(project_name)
    print(project_id)
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("Você deve definir o project_name manualmente na célula de código a seguir")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

model-build-10-15-04-05-48
p-lcexwhfupbld


In [14]:
# project_name = "<INSIRA O NOME DO PROJETO CRIADO>" # Mantenha comentado se você usou a opção 1 para criar um projeto
r = sm.describe_project(ProjectName=project_name)
project_id = r['ProjectId']
project_arn = r['ProjectArn']
project_folder = f"sagemaker-{project_name}-{project_id}-modelbuild"
project_repo_url = f"codecommit::{region}://sagemaker-{project_name}-{project_id}-modelbuild"

print(f"Pasta do projeto: {project_folder}")
print(f"URL do repositório do projeto: {project_repo_url}")

Pasta do projeto: sagemaker-model-build-10-15-04-05-48-p-lcexwhfupbld-modelbuild
URL do repositório do projeto: codecommit::us-east-1://sagemaker-model-build-10-15-04-05-48-p-lcexwhfupbld-modelbuild


### 1. Clone o código inicial do projeto para o sistema de arquivos do JupyterLab
Você precisa clonar o código do projeto do repositório CodeCommit usando o terminal CLI.

1. Abra uma nova janela de terminal via **Arquivo** > **Novo** > **Terminal**
2. Instale o auxiliar `git-remote-codecommit`: ```pip install git-remote-codecommit```
3. Clone o repositório do projeto: ```git clone <URL DO REPOSITÓRIO DO PROJETO>```. Substitua `<URL DO REPOSITÓRIO DO PROJETO>` pela URL real do repositório do projeto obtida na célula de código acima.

### 2. Substituir o código de construção do pipeline


Os seguintes passos são necessários para personalizar o projeto que contém o código do modelo. A próxima célula de código executa todos os passos necessários, você não precisa fazer nada manualmente. O texto a seguir é apenas para sua informação.

- O código-fonte está na pasta `sagemaker-<nome-do-projeto>-<id-do-projeto>-modelbuild`.
- O arquivo original `codebuild-buildspec.yml` é renomeado para `codebuild-buildspec-original.yml`.
- A pasta do repositório de código do projeto contendo o código do pipeline é renomeada de `abalone` para `fromideatoprod`.
- O arquivo original com o pipeline modelo `pipeline.py` é renomeado para `pipeline-original.py`.
- Os módulos Python `pipeline_steps` são copiados para a pasta `pipelines` no diretório do repositório de código do projeto.
- O arquivo `requirements.txt` criado no notebook 3 é copiado para a pasta `pipelines` no diretório do repositório de código do projeto.
- O arquivo de configuração padrão do SDK Python do SageMaker `config.yaml` do notebook 3 é copiado para a pasta `pipelines` no diretório do repositório de código do projeto.

In [15]:
# ver o nome da pasta desse treinamento
!pwd

/home/sagemaker-user/aws-mlops-sagemaker-do-inicio-ao-fim


In [16]:
# se o caminho local para a pasta do treinamento for diferente, defina o caminho absoluto correto para a variável workshop_folder
workshop_folder = "amazon-sagemaker-from-idea-to-production"

In [17]:
!mkdir -p ~/{workshop_folder}/pipelines
!mv ~/{project_folder}/codebuild-buildspec.yml ~/{project_folder}/codebuild-buildspec-original.yml
!mv ~/{project_folder}/pipelines/abalone ~/{project_folder}/pipelines/fromideatoprod
!mv ~/{project_folder}/pipelines/fromideatoprod/pipeline.py ~/{project_folder}/pipelines/fromideatoprod/pipeline-original.py
!cp ~/{workshop_folder}/pipeline_steps/* ~/{project_folder}/pipelines/
!cp ~/{workshop_folder}/pipeline_steps/* ~/{workshop_folder}/pipelines/
!cp ~/{workshop_folder}/requirements.txt ~/{project_folder}
!cp ~/{workshop_folder}/config.yaml ~/{project_folder}

mv: cannot stat '/home/sagemaker-user/sagemaker-model-build-10-15-04-05-48-p-lcexwhfupbld-modelbuild/codebuild-buildspec.yml': No such file or directory
mv: cannot stat '/home/sagemaker-user/sagemaker-model-build-10-15-04-05-48-p-lcexwhfupbld-modelbuild/pipelines/abalone': No such file or directory
mv: cannot stat '/home/sagemaker-user/sagemaker-model-build-10-15-04-05-48-p-lcexwhfupbld-modelbuild/pipelines/fromideatoprod/pipeline.py': No such file or directory
cp: cannot stat '/home/sagemaker-user/amazon-sagemaker-from-idea-to-production/pipeline_steps/*': No such file or directory
cp: cannot stat '/home/sagemaker-user/amazon-sagemaker-from-idea-to-production/pipeline_steps/*': No such file or directory
cp: cannot stat '/home/sagemaker-user/amazon-sagemaker-from-idea-to-production/requirements.txt': No such file or directory
cp: cannot stat '/home/sagemaker-user/amazon-sagemaker-from-idea-to-production/config.yaml': No such file or directory


Execute a seguinte célula para escrever o código de construção do pipeline no arquivo `pipeline.py`. Reutilize o código do notebook do passo 3 como a função `get_pipeline()`.

<div style="border: 4px solid coral; text-align: center; margin: auto;">
    <p style=" text-align: center; margin: auto;">O código de construção do pipeline funciona tanto com o conjunto de dados brutos do S3 quanto com o conjunto de features do Feature Store, você precisa passar um parâmetro de entrada correspondente para o pipeline.
    </p>
</div>

Overwriting pipeline.py


In [26]:
%%writefile pipeline.py

import pandas as pd
import json
import boto3
import pathlib
import io
import os
import sagemaker
import mlflow
from time import gmtime, strftime, sleep
from sagemaker.deserializers import CSVDeserializer
from sagemaker.serializers import CSVSerializer

from sagemaker.workflow.execution_variables import ExecutionVariables
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.xgboost.estimator import XGBoost
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import (
    ProcessingInput, 
    ProcessingOutput, 
    ScriptProcessor
)
from sagemaker.inputs import TrainingInput

from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import (
    ProcessingStep, 
    TrainingStep, 
    CreateModelStep,
    CacheConfig
)
from sagemaker.workflow.check_job_config import CheckJobConfig
from sagemaker.workflow.parameters import (
    ParameterInteger, 
    ParameterFloat, 
    ParameterString, 
    ParameterBoolean
)
from sagemaker.workflow.quality_check_step import (
    DataQualityCheckConfig,
    ModelQualityCheckConfig,
    QualityCheckStep,
)
from sagemaker.workflow.clarify_check_step import (
    ModelBiasCheckConfig, 
    ClarifyCheckStep, 
    ModelExplainabilityCheckConfig
)
from sagemaker import Model
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.model_step import ModelStep
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.conditions import (
    ConditionGreaterThan,
    ConditionGreaterThanOrEqualTo
)
from sagemaker.workflow.parallelism_config import ParallelismConfiguration
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import (
    Join,
    JsonGet
)
from sagemaker.workflow.lambda_step import (
    LambdaStep,
    LambdaOutput,
    LambdaOutputTypeEnum,
)
from sagemaker.lambda_helper import Lambda

from sagemaker.model_metrics import (
    MetricsSource, 
    ModelMetrics, 
    FileSource
)
from sagemaker.drift_check_baselines import DriftCheckBaselines
from sagemaker.workflow.pipeline_definition_config import PipelineDefinitionConfig 
from sagemaker.image_uris import retrieve
from sagemaker.workflow.function_step import step
from sagemaker.workflow.step_outputs import get_step
from sagemaker.model_monitor import DatasetFormat, model_monitoring

from pipeline_steps.preprocess import preprocess
from pipeline_steps.evaluate import evaluate
from pipeline_steps.register import register
from pipeline_steps.extract import prepare_datasets

def get_sagemaker_client(region):
     return boto3.Session(region_name=region).client("sagemaker")

def get_pipeline_session(region, bucket_name):
    """Obtém a sessão do pipeline com base na região.

    Args:
        region: a região da AWS para iniciar a sessão
        bucket_name: o bucket a ser usado para armazenar os artefatos

    Returns:
        Instância da PipelineSession
    """

    boto_session = boto3.Session(region_name=region)
    sagemaker_client = boto_session.client("sagemaker")

    return PipelineSession(
        boto_session=boto_session,
        sagemaker_client=sagemaker_client,
        default_bucket=bucket_name,
    )

def get_pipeline_custom_tags(new_tags, region, sagemaker_project_name=None):
    try:
        print(f"Obtendo tags do projeto para {sagemaker_project_name}")
        
        sm_client = get_sagemaker_client(region)
        
        project_arn = sm_client.describe_project(ProjectName=sagemaker_project_name)['ProjectArn']
        project_tags = sm_client.list_tags(ResourceArn=project_arn)['Tags']

        print(f"Tags do projeto: {project_tags}")
        
        for project_tag in project_tags:
            new_tags.append(project_tag)
            
    except Exception as e:
        print(f"Erro ao obter tags do projeto: {e}")
        
    return new_tags
    
def get_pipeline(
    region,
    sagemaker_project_id=None,
    sagemaker_project_name=None,
    role=None,
    bucket_name=None,
    bucket_prefix="from-idea-to-prod/xgboost",
    input_s3_url=None,
    feature_group_name=None,
    model_package_group_name="from-idea-to-prod-model-group",
    pipeline_name_prefix="from-idea-to-prod-pipeline",
    process_instance_type="ml.m5.large",
    train_instance_type="ml.m5.xlarge",
    test_score_threshold=0.70,
    tracking_server_arn=None,
):
    """Obtém uma instância do SageMaker ML Pipeline.
    
    Returns:
        uma instância de um pipeline
    """
    if feature_group_name is None and input_s3_url is None:
        print("Um dos parâmetros feature_group_name ou input_s3_url deve ser fornecido. Saindo...")
        return None

    session = get_pipeline_session(region, bucket_name)
    sm = session.sagemaker_client
    
    if role is None:
        role = sagemaker.session.get_execution_role(session)

    print(f"versão sagemaker: {sagemaker.__version__}")
    print(f"Função de execução: {role}")
    print(f"URL de entrada S3: {input_s3_url}")
    print(f"Grupo de recursos: {feature_group_name}")
    print(f"Grupo de pacotes de modelos: {model_package_group_name}")
    print(f"Prefixo do nome do pipeline: {pipeline_name_prefix}")
    print(f"ARN do servidor de rastreamento: {tracking_server_arn}")
    
    pipeline_name = f"{pipeline_name_prefix}-{sagemaker_project_id}"
    experiment_name = pipeline_name

    output_s3_prefix = f"s3://{bucket_name}/{bucket_prefix}"
    # Define o URL do S3 de saída para o artefato do modelo
    output_s3_url = f"{output_s3_prefix}/output"
    # Define o URL do S3 de saída para os resultados da consulta do Feature Store
    output_query_location = f'{output_s3_prefix}/offline-store/query_results'
    
    # Define os URLs do S3 de saída para os dados processados
    train_s3_url = f"{output_s3_prefix}/train"
    validation_s3_url = f"{output_s3_prefix}/validation"
    test_s3_url = f"{output_s3_prefix}/test"
    evaluation_s3_url = f"{output_s3_prefix}/evaluation"
    
    baseline_s3_url = f"{output_s3_prefix}/baseline"
    prediction_baseline_s3_url = f"{output_s3_prefix}/prediction_baseline"
    
    xgboost_image_uri = sagemaker.image_uris.retrieve(
            "xgboost", 
            region=region,
            version="1.5-1"
    )

    # Se nenhum ARN do servidor de rastreamento for fornecido, tente encontrar um servidor MLflow ativo
    if tracking_server_arn is None:
        r = sm.list_mlflow_tracking_servers(
            TrackingServerStatus='Created',
        )['TrackingServerSummaries']
    
        if len(r) < 1:
            print("Você não possui nenhum servidor MLflow em execução. Saindo...")
            return None
        else:
            tracking_server_arn = r[0]['TrackingServerArn']
            print(f"Usando o ARN do servidor de rastreamento:{tracking_server_arn}")
        
    # Parâmetros para execução do pipeline
    
    # Define o tipo de instância de processamento
    process_instance_type_param = ParameterString(
        name="ProcessingInstanceType",
        default_value=process_instance_type,
    )

    # Define o tipo de instância de treinamento
    train_instance_type_param = ParameterString(
        name="TrainingInstanceType",
        default_value=train_instance_type,
    )

    # Define o parâmetro de aprovação do modelo
    model_approval_status_param = ParameterString(
        name="ModelApprovalStatus",
        default_value="PendingManualApproval"
    )

    # Threshold mínimo para o desempenho do modelo no conjunto de dados de teste
    test_score_threshold_param = ParameterFloat(
        name="TestScoreThreshold", 
        default_value=test_score_threshold
    )

    # URL do S3 para o conjunto de dados de entrada
    input_s3_url_param = ParameterString(
        name="InputDataUrl",
        default_value=input_s3_url if input_s3_url else "None",
    )

    # Nome do grupo de recursos para o conjunto de recursos de entrada
    feature_group_name_param = ParameterString(
        name="FeatureGroupName",
        default_value=feature_group_name if feature_group_name else "None",
    )
    
    # Nome do grupo de pacotes de modelos
    model_package_group_name_param = ParameterString(
        name="ModelPackageGroupName",
        default_value=model_package_group_name,
    )

    # ARN do servidor de rastreamento do MLflow
    tracking_server_arn_param = ParameterString(
        name="TrackingServerARN",
        default_value=tracking_server_arn,
    )
    
    # Define a configuração de cache de etapas
    cache_config = CacheConfig(
        enable_caching=True,
        expire_after="P30d" # 30 dias
    )

    # Constrói o pipeline
    
    # Obtém os conjuntos de dados
    step_get_datasets = step(
            preprocess, 
            role=role,
            instance_type=process_instance_type_param,
            name=f"preprocess",
            keep_alive_period_in_seconds=3600,
    )(
        input_data_s3_path=input_s3_url_param,
        output_s3_prefix=output_s3_prefix,
        tracking_server_arn=tracking_server_arn_param,
        experiment_name=experiment_name,
        pipeline_run_name=ExecutionVariables.PIPELINE_EXECUTION_ID,
    ) if input_s3_url else step(
        prepare_datasets, 
        role=role,
        instance_type=process_instance_type_param,
        name=f"extract-featureset",
        keep_alive_period_in_seconds=3600,
    )(
        feature_group_name=feature_group_name_param,
        output_s3_prefix=output_s3_prefix,
        query_output_s3_path=output_query_location,
        tracking_server_arn=tracking_server_arn_param,
        experiment_name=experiment_name,
        pipeline_run_name=ExecutionVariables.PIPELINE_EXECUTION_ID,
    )
    
    # Instancia um objeto estimador XGBoost
    estimator = sagemaker.estimator.Estimator(
        image_uri=xgboost_image_uri,
        role=role, 
        instance_type=train_instance_type_param,
        instance_count=1,
        output_path=output_s3_url,
        sagemaker_session=session,
        base_job_name=f"{pipeline_name}-train"
    )
    
    # Define os hiperparâmetros do algoritmo
    estimator.set_hyperparameters(
        num_round=100, # o número de rodadas para executar o treinamento
        max_depth=3, # profundidade máxima de uma árvore
        eta=0.5, # redução do tamanho do passo usada nas atualizações para evitar overfitting
        alpha=2.5, # termo de regularização L1 nos pesos
        objective="binary:logistic",
        eval_metric="auc", # métricas de avaliação para dados de validação
        subsample=0.8, # razão de subsample da instância de treinamento
        colsample_bytree=0.8, # razão de subsample de colunas ao construir cada árvore
        min_child_weight=3, # soma mínima do peso da instância (hessiana) necessária em uma criança
        early_stopping_rounds=10, # o modelo treina até que a pontuação de validação pare de melhorar
        verbosity=1, # verbosidade da impressão de mensagens
    )
    
    # etapa de treinamento
    step_train = TrainingStep(
        name=f"train",
        step_args=estimator.fit(
            {
                "train": TrainingInput(
                    step_get_datasets['train_data'],
                    content_type="text/csv",
                ),
                "validation": TrainingInput(
                    step_get_datasets['validation_data'],
                    content_type="text/csv",
                ),
            }
        ),
        cache_config=cache_config,
    )   
    
    # Etapa de avaliação
    step_evaluate = step(
        evaluate,
        role=role,
        instance_type=process_instance_type_param,
        name=f"evaluate",
        keep_alive_period_in_seconds=3600,
    )(
        test_x_data_s3_path=step_get_datasets['test_x_data'],
        test_y_data_s3_path=step_get_datasets['test_y_data'],
        model_s3_path=step_train.properties.ModelArtifacts.S3ModelArtifacts,
        output_s3_prefix=output_s3_prefix,
        tracking_server_arn=tracking_server_arn_param,
        experiment_name=step_get_datasets['experiment_name'],
        pipeline_run_id=step_get_datasets['pipeline_run_id'],
    )

    # etapa de registro do modelo
    step_register = step(
        register,
        role=role,
        instance_type=process_instance_type_param,
        name=f"register",
        keep_alive_period_in_seconds=3600,
    )(
        training_job_name=step_train.properties.TrainingJobName,
        model_package_group_name=model_package_group_name_param,
        model_approval_status=model_approval_status_param,
        evaluation_result=step_evaluate['evaluation_result'],
        output_s3_prefix=output_s3_url,
        tracking_server_arn=tracking_server_arn_param,
        experiment_name=step_get_datasets['experiment_name'],
        pipeline_run_id=step_get_datasets['pipeline_run_id'],
    )

    # etapa de falha na execução do pipeline
    step_fail = FailStep(
        name=f"fail",
        error_message=Join(on=" ", values=["Execução falhou devido à pontuação AUC < ", test_score_threshold_param]),
    )
    
    # condição para verificar na etapa condicional
    condition_gte = ConditionGreaterThanOrEqualTo(
            left=step_evaluate['evaluation_result']['classification_metrics']['auc_score']['value'],  
            right=test_score_threshold_param,
    )
    
    # etapa de registro condicional
    step_conditional_register = ConditionStep(
        name=f"check-metrics",
        conditions=[condition_gte],
        if_steps=[step_register],
        else_steps=[step_fail],
    )   

    # Cria um objeto de pipeline
    pipeline = Pipeline(
        name=f"{pipeline_name}",
        parameters=[
            input_s3_url_param,
            feature_group_name_param,
            process_instance_type_param,
            train_instance_type_param,
            model_approval_status_param,
            test_score_threshold_param,
            model_package_group_name_param,
            tracking_server_arn_param,
        ],
        steps=[step_conditional_register],
        pipeline_definition_config=PipelineDefinitionConfig(use_custom_job_prefix=True)
    )
    
    return pipeline


Overwriting pipeline.py


Copie este arquivo `pipeline.py` da pasta do workshop para a pasta `pipelines/fromideatoprod` na pasta do repositório de código do projeto.


In [27]:
!cp ~/{workshop_folder}/pipeline.py ~/{project_folder}/pipelines/fromideatoprod/

cp: cannot stat '/home/sagemaker-user/amazon-sagemaker-from-idea-to-production/pipeline.py': No such file or directory


Teste a função `get_pipeline` localmente para verificar se tudo funciona antes de executá-la remotamente.


In [28]:
from pipeline import get_pipeline

In [29]:
# Se você criou a feature store no notebook 3, poderá definir o parâmetro feature_group_name em vez de input_s3_url para obter os dados do 
# armazenamento de recursos
p = get_pipeline(
    region=region,
    sagemaker_project_id=project_id,
    sagemaker_project_name=project_name,
    role=sm_role,
    bucket_name=bucket_name,
    bucket_prefix=bucket_prefix,
    input_s3_url=input_s3_url,
    # feature_group_name=dataset_feature_group_name,
    model_package_group_name=model_package_group_name,
    pipeline_name_prefix=pipeline_name,
    process_instance_type="ml.m5.large",
    train_instance_type="ml.m5.xlarge",
    test_score_threshold=0.70,
    tracking_server_arn=mlflow_arn,
)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


versão sagemaker: 2.232.2
Função de execução: arn:aws:iam::533267005474:role/service-role/AmazonSageMaker-ExecutionRole-20241011T175459
URL de entrada S3: s3://sagemaker-us-east-1-533267005474/from-idea-to-prod/xgboost/input/bank-additional-full.csv
Grupo de recursos: None
Grupo de pacotes de modelos: from-idea-to-prod-pipeline-model-15-02-08-59
Prefixo do nome do pipeline: from-idea-to-prod-pipeline-15-02-08-59
ARN do servidor de rastreamento: arn:aws:sagemaker:us-east-1:533267005474:mlflow-tracking-server/mlflow-d-j2g52sky4w9s-14-22-19-50


/opt/conda/lib/python3.11/site-packages/sagemaker/workflow/pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


In [30]:
p.definition()

sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.Dependencies
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.IncludeLocalWorkDir
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.CustomFileFilter.IgnoreNamePatterns


2024-10-15 04:17:21,557 sagemaker.remote_function INFO     Uploading serialized function code to s3://sagemaker-us-east-1-533267005474/from-idea-to-prod-pipeline-15-02-08-59-p-lcexwhfupbld/register/2024-10-15-04-17-18-612/function
2024-10-15 04:17:21,623 sagemaker.remote_function INFO     Uploading serialized function arguments to s3://sagemaker-us-east-1-533267005474/from-idea-to-prod-pipeline-15-02-08-59-p-lcexwhfupbld/register/2024-10-15-04-17-18-612/arguments
2024-10-15 04:17:21,819 sagemaker.remote_function INFO     Copied dependencies file at './requirements.txt' to '/tmp/tmp97jeqahu/requirements.txt'
2024-10-15 04:17:21,858 sagemaker.remote_function INFO     Successfully uploaded dependencies and pre execution scripts to 's3://sagemaker-us-east-1-533267005474/from-idea-to-prod-pipeline-15-02-08-59-p-lcexwhfupbld/register/2024-10-15-04-17-18-612/pre_exec_script_and_dependencies'
2024-10-15 04:17:21,919 sagemaker.remote_function INFO     Copied user workspace to '/tmp/tmpc210d_gz/

sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.Dependencies
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.IncludeLocalWorkDir
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.CustomFileFilter.IgnoreNamePatterns


2024-10-15 04:17:27,260 sagemaker.remote_function INFO     Uploading serialized function code to s3://sagemaker-us-east-1-533267005474/from-idea-to-prod-pipeline-15-02-08-59-p-lcexwhfupbld/evaluate/2024-10-15-04-17-18-612/function
2024-10-15 04:17:27,329 sagemaker.remote_function INFO     Uploading serialized function arguments to s3://sagemaker-us-east-1-533267005474/from-idea-to-prod-pipeline-15-02-08-59-p-lcexwhfupbld/evaluate/2024-10-15-04-17-18-612/arguments
2024-10-15 04:17:27,438 sagemaker.remote_function INFO     Copied dependencies file at './requirements.txt' to '/tmp/tmpkagwwl8m/requirements.txt'
2024-10-15 04:17:27,484 sagemaker.remote_function INFO     Successfully uploaded dependencies and pre execution scripts to 's3://sagemaker-us-east-1-533267005474/from-idea-to-prod-pipeline-15-02-08-59-p-lcexwhfupbld/evaluate/2024-10-15-04-17-18-612/pre_exec_script_and_dependencies'


sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.Dependencies
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.IncludeLocalWorkDir
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.CustomFileFilter.IgnoreNamePatterns


2024-10-15 04:17:30,382 sagemaker.remote_function INFO     Uploading serialized function code to s3://sagemaker-us-east-1-533267005474/from-idea-to-prod-pipeline-15-02-08-59-p-lcexwhfupbld/preprocess/2024-10-15-04-17-18-612/function
2024-10-15 04:17:30,523 sagemaker.remote_function INFO     Uploading serialized function arguments to s3://sagemaker-us-east-1-533267005474/from-idea-to-prod-pipeline-15-02-08-59-p-lcexwhfupbld/preprocess/2024-10-15-04-17-18-612/arguments
2024-10-15 04:17:30,613 sagemaker.remote_function INFO     Copied dependencies file at './requirements.txt' to '/tmp/tmpkqo1bagt/requirements.txt'
2024-10-15 04:17:30,641 sagemaker.remote_function INFO     Successfully uploaded dependencies and pre execution scripts to 's3://sagemaker-us-east-1-533267005474/from-idea-to-prod-pipeline-15-02-08-59-p-lcexwhfupbld/preprocess/2024-10-15-04-17-18-612/pre_exec_script_and_dependencies'


'{"Version": "2020-12-01", "Metadata": {}, "Parameters": [{"Name": "InputDataUrl", "Type": "String", "DefaultValue": "s3://sagemaker-us-east-1-533267005474/from-idea-to-prod/xgboost/input/bank-additional-full.csv"}, {"Name": "FeatureGroupName", "Type": "String", "DefaultValue": "None"}, {"Name": "ProcessingInstanceType", "Type": "String", "DefaultValue": "ml.m5.large"}, {"Name": "TrainingInstanceType", "Type": "String", "DefaultValue": "ml.m5.xlarge"}, {"Name": "ModelApprovalStatus", "Type": "String", "DefaultValue": "PendingManualApproval"}, {"Name": "TestScoreThreshold", "Type": "Float", "DefaultValue": 0.7}, {"Name": "ModelPackageGroupName", "Type": "String", "DefaultValue": "from-idea-to-prod-pipeline-model-15-02-08-59"}, {"Name": "TrackingServerARN", "Type": "String", "DefaultValue": "arn:aws:sagemaker:us-east-1:533267005474:mlflow-tracking-server/mlflow-d-j2g52sky4w9s-14-22-19-50"}], "PipelineExperimentConfig": {"ExperimentName": {"Get": "Execution.PipelineName"}, "TrialName": {"

In [31]:
p.upsert(role_arn=sm_role)

2024-10-15 04:17:32,520 sagemaker.remote_function INFO     Uploading serialized function code to s3://sagemaker-us-east-1-533267005474/from-idea-to-prod-pipeline-15-02-08-59-p-lcexwhfupbld/register/2024-10-15-04-17-32-520/function
2024-10-15 04:17:32,602 sagemaker.remote_function INFO     Uploading serialized function arguments to s3://sagemaker-us-east-1-533267005474/from-idea-to-prod-pipeline-15-02-08-59-p-lcexwhfupbld/register/2024-10-15-04-17-32-520/arguments
2024-10-15 04:17:32,930 sagemaker.remote_function INFO     Copied dependencies file at './requirements.txt' to '/tmp/tmpi8hoy7js/requirements.txt'
2024-10-15 04:17:32,952 sagemaker.remote_function INFO     Successfully uploaded dependencies and pre execution scripts to 's3://sagemaker-us-east-1-533267005474/from-idea-to-prod-pipeline-15-02-08-59-p-lcexwhfupbld/register/2024-10-15-04-17-32-520/pre_exec_script_and_dependencies'
2024-10-15 04:17:33,000 sagemaker.remote_function INFO     Copied user workspace to '/tmp/tmprk59ilyx/

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:533267005474:pipeline/from-idea-to-prod-pipeline-15-02-08-59-p-lcexwhfupbld',
 'ResponseMetadata': {'RequestId': 'aeac569a-f295-4ad2-a616-58a84ddf8e91',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'aeac569a-f295-4ad2-a616-58a84ddf8e91',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '121',
   'date': 'Tue, 15 Oct 2024 04:17:37 GMT'},
  'RetryAttempts': 0}}

Para ver o pipeline criado na interface do Studio, clique no link construído pela célula de código abaixo:

In [32]:
from IPython.display import HTML

# Mostra o link do pipeline
display(
    HTML('<b>Veja <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/pipelines/{}/graph">o pipeline</a> na interface do Studio</b>'.format(
            domain_id, region, p.describe()['PipelineName']))
)

Neste ponto, você testou localmente se o código de construção do pipeline funciona e ele cria um pipeline. Você pode ver este pipeline no widget **Pipelines** do Studio. Agora você está pronto para criar um pipeline de CI/CD.

#### Anexar o grupo de pacotes de modelos ao projeto
Recursos pertencentes ao projeto são automaticamente marcados com as tags `sagemaker:project-name` e `sagemaker:project-id` para controle de custos, controle de segurança baseado em atributos e governança.
Como o grupo de pacotes de modelos já existe no registro de modelos, você precisa marcá-lo para anexá-lo a este projeto. A célula de código a seguir chama a API `AddTags` para definir tags de projeto no grupo de pacotes de modelos.

In [33]:
model_package_group_arn = sm.describe_model_package_group(ModelPackageGroupName=model_package_group_name).get("ModelPackageGroupArn")

if model_package_group_arn:
    print(f"Adicionando tags {project_arn.split('/')[-1]} e {project_id} para o grupo de pacotes de modelos {model_package_group_arn}")
    r = sm.add_tags(
        ResourceArn=model_package_group_arn,
        Tags=[
            {
                'Key': 'sagemaker:project-name',
                'Value': project_arn.split("/")[-1]
            },
            {
                'Key': 'sagemaker:project-id',
                'Value': project_id
            },
        ]
    )
    print(r)
else:
    print(f"O grupo de pacotes de modelos {model_package_group_name} não existe")
    
sm.list_tags(ResourceArn=model_package_group_arn)["Tags"]

Adicionando tags model-build-10-15-04-05-48 e p-lcexwhfupbld para o grupo de pacotes de modelos arn:aws:sagemaker:us-east-1:533267005474:model-package-group/from-idea-to-prod-pipeline-model-15-02-08-59
{'Tags': [{'Key': 'sagemaker:project-name', 'Value': 'model-build-10-15-04-05-48'}, {'Key': 'sagemaker:project-id', 'Value': 'p-lcexwhfupbld'}], 'ResponseMetadata': {'RequestId': 'b39b6372-8644-415a-a343-b1c21ea625d0', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'b39b6372-8644-415a-a343-b1c21ea625d0', 'content-type': 'application/x-amz-json-1.1', 'content-length': '136', 'date': 'Tue, 15 Oct 2024 04:17:38 GMT'}, 'RetryAttempts': 0}}


[{'Key': 'sagemaker:project-name', 'Value': 'model-build-10-15-04-05-48'},
 {'Key': 'sagemaker:user-profile-arn',
  'Value': 'arn:aws:sagemaker:us-east-1:533267005474:user-profile/d-j2g52sky4w9s/default-20241011T175458'},
 {'Key': 'sagemaker:domain-arn',
  'Value': 'arn:aws:sagemaker:us-east-1:533267005474:domain/d-j2g52sky4w9s'},
 {'Key': 'sagemaker:project-id', 'Value': 'p-lcexwhfupbld'},
 {'Key': 'sagemaker:space-arn',
  'Value': 'arn:aws:sagemaker:us-east-1:533267005474:space/d-j2g52sky4w9s/MLOpsCodeSpace'}]

### 3. Modificar o arquivo de especificação de build
Agora, modifique o arquivo `codebuild-buildspec.yml` na pasta do projeto para refletir o novo nome do módulo Python com seu pipeline e definir outros parâmetros específicos do projeto.

Você precisa passar os seguintes parâmetros para um script de criação de pipeline — correspondentes aos parâmetros da função `get_pipeline` que você acabou de criar:

- `input_s3_url` - um URL do S3 para o conjunto de dados brutos de entrada. Se você criou um grupo de recursos no notebook 3, você pode usar o parâmetro `feature_group_name`  em vez disso
- `feature_group_name` –você pode usar este parâmetro se criou este grupo de recursos no notebook 3, neste caso você não precisa fornecer input_s3_url
- `model_package_group_name` – o pacote do registro de modelos para registrar um modelo após o treinamento
- `pipeline_name_prefix` – um prefixo de nome para o pipeline. O nome do pipeline é construído como `<pipeline_name_prefix>-<project-id>`
- `role` –a função de execução do pipeline
- `tracking_server_arn`- o ARN do servidor MLflow para rastreamento de execução do pipeline

As células a seguir imprimem os valores desses parâmetros:

In [34]:
try:
    print(f"""
        INPUT-S3-URL: {input_s3_url}
        FEATURE-GROUP-NAME: {dataset_feature_group_name}
        MODEL-PACKAGE-GROUP-NAME: {model_package_group_name}
        PIPELINE-BASE-NAME: {pipeline_name}
        SAGEMAKER-EXECUTION-ROLE-ARN: {sm_role}
        TRACKING-SERVER-ARN: {mlflow_arn}
        """)
except NameError:
    print(f"""
        O nome do grupo de recursos do conjunto de dados não está definido, use input_s3_url em vez disso.:
        ********************************************************************
        
        INPUT-S3-URL: {input_s3_url}
        MODEL-PACKAGE-GROUP-NAME: {model_package_group_name}
        PIPELINE-BASE-NAME: {pipeline_name}
        SAGEMAKER-EXECUTION-ROLE-ARN: {sm_role}
        TRACKING-SERVER-ARN: {mlflow_arn}
        """)


        INPUT-S3-URL: s3://sagemaker-us-east-1-533267005474/from-idea-to-prod/xgboost/input/bank-additional-full.csv
        FEATURE-GROUP-NAME: from-idea-to-prod-15-02-45-15
        MODEL-PACKAGE-GROUP-NAME: from-idea-to-prod-pipeline-model-15-02-08-59
        PIPELINE-BASE-NAME: from-idea-to-prod-pipeline-15-02-08-59
        SAGEMAKER-EXECUTION-ROLE-ARN: arn:aws:iam::533267005474:role/service-role/AmazonSageMaker-ExecutionRole-20241011T175459
        TRACKING-SERVER-ARN: arn:aws:sagemaker:us-east-1:533267005474:mlflow-tracking-server/mlflow-d-j2g52sky4w9s-14-22-19-50
        


Agora, substitua o valor desses parâmetros na célula de código a seguir pelos valores impressos na célula acima.

Para fazer isso, localize o parâmetro `kwargs` na célula de código a seguir, começando com `%%writefile codebuild-buildspec.yml`:

```
--kwargs "{ \
    \"input_s3_url\":\"<INPUT-S3-URL>\", \
    \"feature_group_name\":\"<FEATURE-GROUP-NAME>\", \
    \"model_package_group_name\":\"<MODEL-PACKAGE-GROUP-NAME>\", \
    \"pipeline_name_prefix\":\"<PIPELINE-BASE-NAME>\", \
    \"role\":\"<SAGEMAKER-EXECUTION-ROLE-ARN>\", \
    \"tracking_server_arn\":\"<TRACKING-SERVER-ARN>\", \
    \"region\":\"${AWS_REGION}\", \
    \"sagemaker_project_name\":\"${SAGEMAKER_PROJECT_NAME}\", \
    \"sagemaker_project_id\":\"${SAGEMAKER_PROJECT_ID}\", \
    \"bucket_name\":\"${ARTIFACT_BUCKET}\" \
        }"
```

e substitua os valores dos parâmetros `input_s3_url` OU `feature_group_name`, `model_package_group_name`, `pipeline_name_prefix`, `role` e `tracking_server_arn` pelos valores impressos pelas células anteriores.

Você precisa substituir apenas um de `input_s3_url` ou `feature_group_name`, dependendo do método de entrada de dados que você deseja usar — um conjunto de dados brutos de entrada do S3 ou o conjunto de recursos processado do Feature Store. Você pode usar o Feature Store apenas se o criou no notebook anterior.

<div class="alert alert-info">Apague a linha com o parâmetro que você não usa: `input_s3_url` ou `feature_group_name` do código da célula.</div>

![](img/codebuild-buildspec-edit.png)

Depois de substituir os valores do parâmetro, execute a célula para gravar um arquivo de especificação de build.


In [35]:
%%writefile codebuild-buildspec.yml

version: 0.2

phases:
  install:
    runtime-versions:
      python: 3.10
    commands:
      - pip install --upgrade --force-reinstall . "awscli>1.20.30"
      - pip install mlflow==2.13.2 sagemaker-mlflow s3fs xgboost
    
  build:
    commands:
      - export SAGEMAKER_USER_CONFIG_OVERRIDE="./config.yaml"
      - export PYTHONUNBUFFERED=TRUE
      - export SAGEMAKER_PROJECT_NAME_ID="${SAGEMAKER_PROJECT_NAME}-${SAGEMAKER_PROJECT_ID}"
      - |
        run-pipeline --module-name pipelines.fromideatoprod.pipeline \
          --role-arn $SAGEMAKER_PIPELINE_ROLE_ARN \
          --tags "[{\"Key\":\"sagemaker:project-name\",\"Value\":\"${SAGEMAKER_PROJECT_NAME}\"}, {\"Key\":\"sagemaker:project-id\", \"Value\":\"${SAGEMAKER_PROJECT_ID}\"}]" \
          --kwargs "{ \
                \"input_s3_url\":\"s3://sagemaker-us-east-1-398333718380/from-idea-to-prod/xgboost/input/bank-additional-full.csv\", \
                \"model_package_group_name\":\"from-idea-to-prod-pipeline-model-23-07-51-02\",\
                \"pipeline_name_prefix\":\"from-idea-to-prod-pipeline-23-07-51-02\",\
                \"role\":\"arn:aws:iam::398333718380:role/cfnstudiodomain-SageMakerExecutionRole-8XQVuLUFsn6g\",\
                \"tracking_server_arn\":\"arn:aws:sagemaker:us-east-1:398333718380:mlflow-tracking-server/mlflow-d-wxmppmljqkj5\", \
                \"region\":\"${AWS_REGION}\", \
                \"sagemaker_project_name\":\"${SAGEMAKER_PROJECT_NAME}\",\
                \"sagemaker_project_id\":\"${SAGEMAKER_PROJECT_ID}\",\
                \"bucket_name\":\"${ARTIFACT_BUCKET}\"\
                    }"
      - echo "Criação/atualização do SageMaker Pipeline e execução do pipeline concluída.."

Writing codebuild-buildspec.yml


Copie o arquivo `codebuild-buildspec.yml` da pasta do treinamento para a pasta do repositório de código do projeto.


In [36]:
!cp ~/{workshop_folder}/codebuild-buildspec.yml ~/{project_folder}/codebuild-buildspec.yml

cp: cannot stat '/home/sagemaker-user/amazon-sagemaker-from-idea-to-production/codebuild-buildspec.yml': No such file or directory


Para resumir, você acabou de fazer três alterações no arquivo de especificação de build:

1. Modificou o valor do parâmetro `--module-name` de `run-pipeline` de `pipelines.abalone.pipeline` para o novo caminho `pipelines.fromideatoprod.pipeline`
2. Removeu alguns parâmetros da lista `kwargs` para usar os valores padrão do parâmetro da função `get_pipeline()`
3. Adicionou parâmetros adicionais para o pipeline à lista de parâmetros `kwargs`



### 4. Corrigir o arquivo `setup.py`

Finalmente, abra o arquivo `setup.py` na pasta do repositório de código do projeto e substitua a linha `required_packages = ["sagemaker==2.XX.0"]` por `required_packages = ["sagemaker"]`. Salve suas alterações.

Por que você fez essa alteração? A versão fixa da biblioteca sagemaker é um bug e será corrigida em futuras versões dos modelos de projeto SageMaker integrados. Por enquanto, você corrige este arquivo de modelo manualmente. Lembre-se de que os modelos de projeto integrados são apenas para sua conveniência e para demonstrar como usar o mecanismo de projeto SageMaker para empacotar e provisionar seus próprios projetos personalizados de MLOps.

Agora você está pronto para lançar o pipeline de construção de modelos de CI/CD.

Tudo está pronto para executar um pipeline de CI/CD.


---

## Executar o CI/CD para o pipeline de construção do modelo

Para iniciar o CI/CD para o pipeline de construção do modelo, você precisa enviar o código alterado para o repositório CodeCommit do projeto.

<div class="alert alert-info">Certifique-se de que você está na pasta que contém o código do repositório no terminal JupyterLab ao executar comandos git. O nome da pasta é semelhante a `sagemaker-[project-name]-[project-id]-modelbuild`.</div>

A célula a seguir imprime o comando `cd` necessário com o nome de pasta correto:



In [37]:
print(f"cd ~/{project_folder}")

cd ~/sagemaker-model-build-10-15-04-05-48-p-lcexwhfupbld-modelbuild


Abra uma janela de terminal do sistema por meio do menu JupyterLab **Arquivo** > **Novo** > **Terminal** e insira os comandos a seguir. Mantenha `user.email` e `user.name` ou substitua pelos seus dados.

```sh
cd ~/<PROJECT-FOLDER>/<PROJECT-CODE-REPOSITORY-FOLDER>

git config --global user.email "you@example.com"
git config --global user.name "Your Name"

git add -A
git commit -am "personalizar projeto"
git push
```

Após enviar suas alterações de código, o projeto inicia uma execução do pipeline CodePipeline que constrói, certifica e executa o pipeline de construção de modelos SageMaker. Essa nova execução do pipeline cria uma nova versão do modelo no grupo de pacotes de modelos no registro de modelos SageMaker.

Você pode acompanhar a execução do pipeline no widget **Pipelines** do Studio.

Aguarde até que a execução do pipeline termine. A execução leva cerca de 15 minutos para ser concluída.



Para ver a execução do pipeline, clique no link construído pela célula de código abaixo. Observe que o CodeBuild leva cerca de 1 minuto para construir, atualizar e iniciar a execução. Atualize a página da interface do usuário do Studio para ver a execução iniciada.


In [39]:
# Mostre o link da pipeline de execução
display(
    HTML('<b>Veja <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/pipelines/{}/executions/">a pipeline de execução</a> no Studio UI</b>'.format(
            domain_id, region, p.describe()['PipelineName']))
)

## Visualizar os detalhes de uma nova versão do modelo

Após a conclusão da execução do pipeline, uma nova versão do modelo deve ser registrada no registro de modelos. Para ver os detalhes da versão do modelo:

1. Na barra lateral do Studio, escolha o widget **Modelos**
2. Clique no nome do grupo de pacotes de modelos que você criou no notebook da etapa 3 (`from-idea-to-prod-pipeline-model-<TIMESTAMP>`) para abrir o grupo de modelos
3. Na lista de versões do modelo, selecione a versão mais recente do modelo

Na guia de versão do modelo que se abre, você pode navegar pela atividade, [detalhes da versão do modelo](https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry-details.html) e [linhagem de dados](https://docs.aws.amazon.com/sagemaker/latest/dg/lineage-tracking.html).

![](img/model-version-details.png)

Em um projeto do mundo real, você adiciona vários atributos de modelo e metadados adicionais da versão do modelo, como [métricas de qualidade do modelo](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-model-quality-metrics.html), relatórios de [explicabilidade](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-model-explainability.html) e [viés](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-measure-post-training-bias.html), dados de teste de carga e [recomendador de inferência](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-recommender.html).


Para visualizar a versão do pacote de modelo na interface do usuário do Studio, clique no link criado pela célula de código abaixo. Observe que você precisa esperar até que a execução do pipeline termine para visualizar a versão mais recente do pacote de modelo registrada.


In [40]:
# Mostrar o link do pacote do modelo
display(
    HTML('<b>Veja <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/models/registered-models/{}/versions">as versões do pacote do modelo</a> no Studio UI</b>'.format(
            domain_id, region, model_package_group_name))
)

## Resumo

Neste notebook, você implementou um pipeline de CI/CD com os seguintes recursos:

- O pipeline de ML de construção de modelos está sob controle de versão em um repositório CodeCommit
- Cada envio para o repositório CodeCommit inicia uma nova compilação do CodeBuild que constrói, atualiza e executa o pipeline de ML
- Todo o processo de desenvolvimento de modelos e2e agora está automatizado, incluindo o pipeline de construção de modelos
- O projeto SageMaker é uma construção lógica no Studio que contém os metadados sobre pipelines de ML, repositórios, modelos, experimentos e endpoints de inferência relacionados



---

## Continue com a etapa 5
Abra o notebook da etapa 5 [notebook](05-deploy.ipynb).


## Ideias de desenvolvimento adicional para seus projetos do mundo real

- Você pode usar um [modelo MLOps fornecido pela SageMaker para construção, treinamento e implantação de modelos com repositórios Git de terceiros usando Jenkins](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-templates-sm.html#sagemaker-projects-templates-git-jenkins)
- Crie um [modelo de projeto SageMaker personalizado](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-templates-custom.html) para atender aos requisitos específicos do seu projeto



## Recursos adicionais

- [Laboratório de Pipelines do Amazon SageMaker no SageMaker Immersion Day](https://catalog.us-east-1.prod.workshops.aws/workshops/63069e26-921c-4ce1-9cc7-dd882ff62575/en-US/lab6)
- [Melhore seu desenvolvimento de aprendizado de máquina usando uma arquitetura modular com projetos do Amazon SageMaker](https://aws.amazon.com/blogs/machine-learning/enhance-your-machine-learning-development-by-using-a-modular-architecture-with-amazon-sagemaker-projects/)
- [Mergulhe fundo na automação do MLOps](https://www.youtube.com/watch?v=3_cHnk9VSfQ)
- [Percurso do projeto SageMaker MLOps](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-walkthrough.html)
- [Repositório GitHub `aws-samples` com exemplos de modelos de projetos personalizados](https://github.com/aws-samples/sagemaker-custom-project-templates)



# Desligar o kernel

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>